In [256]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error

import holidays
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings('ignore')

In [257]:
train = pd.read_csv(r'./data/train.csv')
international_trade = pd.read_csv(r'./data/international_trade.csv')
test = pd.read_csv(r'./data/test.csv')

In [258]:
def group_season(df):
    df.loc[(df['month'] == 3) | (df['month'] == 4) | (df['month'] == 5), 'season'] = '봄'
    df.loc[(df['month'] == 6) | (df['month'] == 7) | (df['month'] == 8), 'season'] = '여름'
    df.loc[(df['month'] == 9) | (df['month'] == 10) | (df['month'] == 11), 'season'] = '가을'
    df.loc[(df['month'] == 12) | (df['month'] == 1) | (df['month'] == 2), 'season'] = '겨울'
    return df['season']

def holiday(df):
    kr_holidays = holidays.KR()
    df['holiday'] = df.timestamp.apply(lambda x: 'holiday' if x in kr_holidays else 'non-holiday')
    return df['holiday']

def cyclical_feature(df, time=12):
    df['sin_time'] = np.sin(2*np.pi*df.month/time)
    df['cos_time'] = np.cos(2*np.pi*df.month/time)

In [259]:
train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))
train['Weekday'] = pd.to_datetime(train['timestamp']).dt.weekday
train['is_weekend'] = train['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
train['total_item_value'] = train['item']+train['corporation']+train['location']
train['location_month'] = train['location'].astype(str) + "_" + train['month'].astype(str)
cyclical_feature(train)

test['year'] = test['timestamp'].apply(lambda x : int(x[0:4]))
test['month'] = test['timestamp'].apply(lambda x : int(x[5:7]))
test['day'] = test['timestamp'].apply(lambda x : int(x[8:10]))
test['Weekday'] = pd.to_datetime(test['timestamp']).dt.weekday
test['is_weekend'] = test['Weekday'].apply(lambda x: 1 if x >= 5 else 0)
test['total_item_value'] = test['item']+test['corporation']+test['location']
test['location_month'] = test['location'].astype(str) + "_" + test['month'].astype(str)
cyclical_feature(test)


train['season'] = group_season(train)
test['season'] = group_season(test)

train['holiday'] = holiday(train)
test['holiday'] = holiday(test)



Q1 = train['price(원/kg)'].quantile(0.25)
Q3 = train['price(원/kg)'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

train = train[(train['price(원/kg)'] >= lower_bound) & (train['price(원/kg)'] <= upper_bound)]



x = train.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)'])
y = train['price(원/kg)']

x_test = test.drop(columns=['ID', 'timestamp'])

qual_col = ['item', 'corporation', 'location', 'season', 'holiday', 'total_item_value', 'location_month']

for i in qual_col:      
    le = LabelEncoder()
    x[i]=le.fit_transform(x[i])
    x_test[i]=le.transform(x_test[i])

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1103)

In [260]:
xgb = XGBRegressor()
xgb.fit(x_train, y_train)
xgb_pred = xgb.predict(x_val)
rmse = mean_squared_error(y_val, xgb_pred, squared=False)
rmse

381.12945822777385

In [305]:
def cross_validate_models(folds, X, y, model):

    models = []

    for idx, (train_idx, val_idx) in enumerate(folds):
        print(f'===================================={idx+1}============================================')
        x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(x_train, y_train, early_stopping_rounds=50,
                  eval_metric='rmse', eval_set=[(x_val, y_val)], verbose=False)

        y_pred = model.predict(x_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        print(f"{idx + 1} Fold RMSE = {rmse}")
        models.append(model)
        print(f'================================================================================\n\n')

    return models

model = XGBRegressor(n_estimators=10000)
folds = list(KFold(n_splits=10, shuffle=True, random_state=1103).split(x, y))
models = cross_validate_models(folds, x, y, model)
predictions = sum(model.predict(x_test) for model in models) / len(folds)

====================================1============================================
1 Fold RMSE = 360.2968048014292


====================================2============================================
2 Fold RMSE = 377.9410013597974


====================================3============================================
3 Fold RMSE = 371.76160920258855


====================================4============================================
4 Fold RMSE = 395.28358820009777


====================================5============================================
5 Fold RMSE = 374.92560742678535


====================================6============================================
6 Fold RMSE = 367.6027294983374


====================================7============================================
7 Fold RMSE = 380.8767938417204


====================================8============================================
8 Fold RMSE = 405.60715674728027


====================================9===============================

In [309]:
submission = pd.read_csv(r'data\sample_submission.csv')
submission['answer'] = np.round(predictions)
submission.to_csv('./xgb_submission.csv', index=False)